In [1]:
_ = !pip3 install -r requirements.txt

In [2]:
%load_ext tensorboard

In [3]:
%tensorboard --logdir logs/nasnet/ --port=6006 --bind_all

In [4]:
from utils import plot_learning_curves, build_data

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [6]:
import sklearn as skl
from sklearn.metrics import f1_score, accuracy_score

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *

In [8]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
train = build_data('train')
val = build_data('val')

In [10]:
EPOCHS = 100
BATCH_SIZE = 32

In [11]:
input_shape = (224, 224, 3)

In [12]:
generator = ImageDataGenerator(
    rotation_range=10, # rotation
    width_shift_range=0.2, # horizontal shift
    height_shift_range=0.2, # vertical shift
    zoom_range=0.2, # zoom
    horizontal_flip=True, # horizontal flip
    validation_split = .15
)
train_gen = generator.flow_from_dataframe(
    dataframe = train,
    x_col = 'image_path',
    y_col = ['valence', 'arousal', 'expression'],
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'multi_output',
    subset = 'training',
    batch_size = BATCH_SIZE,
    preprocessing_function = keras.applications.nasnet.preprocess_input,
    shuffle = True
)

val_gen = generator.flow_from_dataframe(
    dataframe = train,
    x_col = 'image_path',
    y_col = ['valence', 'arousal', 'expression'],
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'multi_output',
    subset = 'validation',
    batch_size = BATCH_SIZE,
    preprocessing_function = keras.applications.nasnet.preprocess_input,
    shuffle = True
)

Found 244504 validated image filenames.
Found 43147 validated image filenames.


In [13]:
test_gen = ImageDataGenerator().flow_from_dataframe(
    dataframe = val,
    x_col = 'image_path',
    y_col = ['valence', 'arousal', 'expression'],
    target_size = (224, 224),
    color_mode = 'rgb',
    class_mode = 'multi_output',
    batch_size = BATCH_SIZE,
    preprocessing_function = keras.applications.nasnet.preprocess_input,
    shuffle = True
)

Found 3999 validated image filenames.


In [14]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_expression_output_loss', 
        patience = 1,
        restore_best_weights = True
    ),
    keras.callbacks.ModelCheckpoint(
        'models/NasNetMobileAffect.h5',
        monitor="val_expression_output_loss",
        save_weights_only=False,
        save_best_only=True,
        verbose=1,
        mode="auto",
        save_freq="epoch",
        period = 1
    )
]

In [15]:
finetuning_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor = 'val_expression_output_loss', 
        patience = 3,
        restore_best_weights = True
    ),
    keras.callbacks.ModelCheckpoint(
        'models/NasNetMobileAffect.h5',
        monitor="val_expression_output_loss",
        save_weights_only=False,
        save_best_only=True,
        verbose=1,
        mode="auto",
        save_freq="epoch",
        period = 1
    ),
    keras.callbacks.TensorBoard(log_dir="logs/nasnet/", update_freq='epoch', histogram_freq = 5)
]

In [16]:
def make_nasmobile():
    inp = Input(input_shape)
    
    base = keras.applications.NASNetMobile(
        include_top=False,
        weights="imagenet",
        input_tensor=inp,
        pooling='avg',
    )
    
    base.trainable = False
    
    embed = Dense(256, kernel_regularizer = 'l2')(base.output)
    embed = BatchNormalization()(embed)
    embed = ReLU()(embed)
    embed = Dropout(.5)(embed)
    
    val_dense = Dense(128, kernel_regularizer = 'l2')(embed)
    val_dense = BatchNormalization()(val_dense)
    val_dense = ReLU()(val_dense)
    val_dense = Dropout(.5)(val_dense)

    aro_dense = Dense(128, kernel_regularizer = 'l2')(embed)
    aro_dense = BatchNormalization()(aro_dense)
    aro_dense = ReLU()(aro_dense)
    aro_dense = Dropout(.5)(aro_dense)
    
    exp_preoutput = Concatenate()([val_dense, aro_dense])
    exp_preoutput = BatchNormalization()(exp_preoutput)
    exp_preoutput = Add()([embed, exp_preoutput])
    exp_preoutput = BatchNormalization()(exp_preoutput)
    
    val_out = Dense(1, activation=None, name = 'valence_output')(val_dense)
    aro_out = Dense(1, activation=None, name = 'arousal_output')(aro_dense)
    exp_out = Dense(8, activation="softmax", name = 'expression_output')(exp_preoutput)
    
    model = keras.Model(inputs = inp, outputs = [val_out, aro_out, exp_out])
    
    return model, base

In [17]:
model, base = make_nasmobile()

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           stem_bn1[0][0]                   
______________________________________________________________________________________________

# Transfer Learning

In [19]:
model.compile(loss=["mse", "mse", "sparse_categorical_crossentropy"],
                  metrics = {
                      'valence_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'arousal_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'expression_output':['accuracy']},
                  optimizer='adam')

In [20]:
history = model.fit(
        x = train_gen,
        epochs = EPOCHS,
        validation_data = val_gen,
        verbose = 1,
        callbacks = callbacks
    )

Epoch 1/100
7641/7641 [==============================] - 1714s 223ms/step - loss: 3.3720 - valence_output_loss: 0.3821 - arousal_output_loss: 0.2601 - expression_output_loss: 1.5959 - valence_output_root_mean_squared_error: 0.6075 - arousal_output_root_mean_squared_error: 0.4792 - expression_output_accuracy: 0.4324 - val_loss: 1.9102 - val_valence_output_loss: 0.2593 - val_arousal_output_loss: 0.0922 - val_expression_output_loss: 1.4793 - val_valence_output_root_mean_squared_error: 0.5092 - val_arousal_output_root_mean_squared_error: 0.3037 - val_expression_output_accuracy: 0.4640

Epoch 00001: val_expression_output_loss improved from inf to 1.47933, saving model to models/NasNetMobileAffect.h5
Epoch 2/100
7641/7641 [==============================] - 1704s 223ms/step - loss: 1.9050 - valence_output_loss: 0.2650 - arousal_output_loss: 0.0927 - expression_output_loss: 1.4723 - valence_output_root_mean_squared_error: 0.5148 - arousal_output_root_mean_squared_error: 0.3044 - expression_out

# Finetuning

In [21]:
base.trainable = True

In [22]:
model.compile(loss=["mse", "mse", "sparse_categorical_crossentropy"],
                  metrics = {
                      'valence_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'arousal_output':[tf.keras.metrics.RootMeanSquaredError()],
                      'expression_output':['accuracy']},
                  optimizer=keras.optimizers.Adam(1e-5))

In [23]:
history_finetune = model.fit(
        x = train_gen,
        epochs = EPOCHS,
        validation_data = val_gen,
        callbacks = finetuning_callbacks,
        verbose = 1
    )

Epoch 1/100
7641/7641 [==============================] - 2012s 261ms/step - loss: 1.5782 - valence_output_loss: 0.2061 - arousal_output_loss: 0.0910 - expression_output_loss: 1.2632 - valence_output_root_mean_squared_error: 0.4532 - arousal_output_root_mean_squared_error: 0.3017 - expression_output_accuracy: 0.5647 - val_loss: 1.1946 - val_valence_output_loss: 0.1118 - val_arousal_output_loss: 0.0893 - val_expression_output_loss: 0.9830 - val_valence_output_root_mean_squared_error: 0.3344 - val_arousal_output_root_mean_squared_error: 0.2988 - val_expression_output_accuracy: 0.6839

Epoch 00001: val_expression_output_loss improved from inf to 0.98305, saving model to models/NasNetMobileAffect.h5
Epoch 2/100
7641/7641 [==============================] - 1993s 261ms/step - loss: 1.1735 - valence_output_loss: 0.1228 - arousal_output_loss: 0.0870 - expression_output_loss: 0.9540 - valence_output_root_mean_squared_error: 0.3504 - arousal_output_root_mean_squared_error: 0.2950 - expression_out


Epoch 00012: val_expression_output_loss improved from 0.67934 to 0.67804, saving model to models/NasNetMobileAffect.h5
Epoch 13/100
7641/7641 [==============================] - 1989s 260ms/step - loss: 0.8323 - valence_output_loss: 0.0943 - arousal_output_loss: 0.0732 - expression_output_loss: 0.6573 - valence_output_root_mean_squared_error: 0.3070 - arousal_output_root_mean_squared_error: 0.2706 - expression_output_accuracy: 0.7732 - val_loss: 0.8341 - val_valence_output_loss: 0.0826 - val_arousal_output_loss: 0.0649 - val_expression_output_loss: 0.6790 - val_valence_output_root_mean_squared_error: 0.2875 - val_arousal_output_root_mean_squared_error: 0.2548 - val_expression_output_accuracy: 0.7617

Epoch 00013: val_expression_output_loss did not improve from 0.67804
Epoch 14/100
7641/7641 [==============================] - 2003s 262ms/step - loss: 0.8300 - valence_output_loss: 0.0950 - arousal_output_loss: 0.0733 - expression_output_loss: 0.6541 - valence_output_root_mean_squared_err

In [24]:
model.save('models/NasNetMobileAffect.h5')

In [25]:
model.evaluate(test_gen)

125/125 [==============================] - 5s 38ms/step - loss: 1.9933 - valence_output_loss: 0.1817 - arousal_output_loss: 0.1456 - expression_output_loss: 1.6583 - valence_output_root_mean_squared_error: 0.4263 - arousal_output_root_mean_squared_error: 0.3816 - expression_output_accuracy: 0.4816


[1.993270993232727,
 0.18172939121723175,
 0.145588219165802,
 1.6582655906677246,
 0.42629730701446533,
 0.38156023621559143,
 0.4816204011440277]

In [26]:
# !tensorflowjs_converter --input_format=keras --metadata= --output_format=tfjs_graph_model --quantize_float16=* --weight_shard_size_bytes=4194304 models/NasNetMobileAffect.h5 models/js/NasNetMobileAffect_Graph